# Surgery

Move all the weight and biases from MikeNet to TensorFlow

In [ ]:
import troubleshooting, meta, modeling
from importlib import reload
import numpy as np
import tensorflow as tf
reload(troubleshooting)
mn_weight = troubleshooting.MikeNetWeight("mikenet/Reading_Weight_v1")  # They have v1-v10, looks similar to me

## Shapes of weights in MikeNet model

In [ ]:
mn_weight.create_weights_shapes()

# Create config file

In [ ]:
code_name = "surgery"
batch_name = None
tf_root = "/home/jupyter/triangle_model"

# Model configs
ort_units = 364
pho_units = 200
sem_units = 2446
hidden_os_units = 500
hidden_op_units = 500
hidden_ps_units = 300
hidden_sp_units = 300
pho_cleanup_units = 50
sem_cleanup_units = 50
pho_noise_level = 0.
sem_noise_level = 0.
activation = "sigmoid"

tau = 1 / 3
max_unit_time = 4.0
output_ticks = 13
inject_error_ticks = 11

# Training configs
learning_rate = 0.005
zero_error_radius = 0.1
save_freq = 20

# Environment configs
tasks = ("pho_sem", "sem_pho", "pho_pho", "sem_sem", "ort_pho", "ort_sem", "triangle")
wf_compression = "log"
wf_clip_low = 0
wf_clip_high = 999_999_999
oral_start_pct = 1.0
oral_end_pct = 1.0

oral_sample = 1_800_000
# oral_tasks_ps = (0.4, 0.4, 0.1, 0.1, 0.)
oral_tasks_ps = (0.4, 0.4, 0.05, 0.15, 0., 0., 0.)
transition_sample = 800_000
reading_sample = 15_000_000
# reading_tasks_ps = (0.2, 0.2, 0.05, 0.05, 0.5)
reading_tasks_ps = (0.2, 0.2, 0.05, 0.05, .1, .1, .3)

batch_size = 100
rng_seed = 2021

## Build model

In [ ]:
cfg = meta.ModelConfig.from_global(globals_dict=globals())
model = modeling.MyModel(cfg)
model.build()

# Grafting

In [ ]:
for weight in model.weights:
    try:
        name = weight.name[:-2]
        weight.assign(mn_weight.weights_tf[name])
        print(f"Grafted mikenet weight {name} to tf.weights")

        # Post-load weight sanity check
        tf.debugging.assert_equal(mn_weight.weights_tf[name], weight)

    except KeyError:
        print(f"Missing weight {name} in mikenet")
        pass



# Parse MN pattern into TF format

In [ ]:
import data_wrangling
x = data_wrangling.load_testset('train_r100')
r100_items = x['item']

In [ ]:
def get_pattern(lines, line_start, line_end):
    pattern = []
    for i in range(line_start, line_end):
        line = lines[i].strip().split(' ')
        [pattern.append(unit) for unit in line]
    return pattern


def parse_pattern(file: str=None) -> dict:
    items = list()
    ort_pattern = {}
    pho_pattern = {}
    sparse_sem_pattern = {}

    if file is None:
        file = "mikenet/englishdict_randcon.pat.txt"

    with open(file, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            line = line.strip()
            if line.startswith("TAG Word: "):
                content = line.split(" ")
                word = content[2]
                ort = content[6] # human readable ort
                pho = content[4] # human readable pho
                print(f"word:{word}, ort:{ort}, pho:{pho}")

                # Record item
                items.append(word)

            if line.startswith("CLAMP Ortho"):
                # Take the next 14 lines value into ort representation dictionary
                ort_pattern[word] = get_pattern(lines, i+1, i+15)

            if line.startswith("TARGET Phono"):
                # Take the next 8 lines value into pho representation dictionary
                pho_pattern[word] = get_pattern(lines, i+1, i+9)

            if line.startswith("TARGET Semantics"):
                sparse_sem_pattern[word] = get_pattern(lines, i+1, i+1)

    return items, ort_pattern, pho_pattern, sparse_sem_pattern

def sparse_to_dense(representation, units=2446) -> np.array:
    """Convert sparse representation to dense representation"""
    dense = np.zeros(units)
    for unit in representation:
        dense[int(unit)] = 1
    return dense

                

In [ ]:
items, ort_pattern, pho_pattern, sparse_sem_pattern = parse_pattern()

In [ ]:
# Pack into my TF data format
# Important: the order of the items must be the same as the order of the inputs
# I did not account for multi meaning items

mn_r100 = {}
mn_r100['item'] = r100_items

np_ort = np.zeros(shape=(100, ort_units))
np_pho = np.zeros(shape=(100, pho_units))
np_sem = np.zeros(shape=(100, sem_units))

for idx, item in enumerate(r100_items):
    np_ort[idx,:] = np.array(ort_pattern[item])
    np_pho[idx,:] = np.array(pho_pattern[item])
    np_sem[idx,:] = sparse_to_dense(sparse_sem_pattern[item], sem_units)

mn_r100['ort'] = np_ort
mn_r100['pho'] = np_pho
mn_r100['sem'] = np_sem
mn_r100['cond'] = None

data_wrangling.save_testset(mn_r100, 'dataset/testsets/mn_r100.pkl.gz')

# Input

In [ ]:
model.set_active_task('triangle')
pred = model([mn_r100['ort']]* 13)


In [ ]:

d = troubleshooting.Diagnosis(code_name)
d.model = model
ts = TestSet_MN(cfg)
ts.eval(testset_name='mn_r100', task='triangle')



In [ ]:

d = troubleshooting.Diagnosis(code_name)
d.eval(testset_name, task='triangle', epoch=200)

@interact(
    sel_word=d.testset_package['item'], 
    layer=['pho', 'sem'], 
    task=['triangle', 'ort_pho', 'exp_osp', 'ort_sem', 'exp_ops'], 
    epoch=(10, d.cfg.total_number_of_epoch + 1, d.cfg.save_freq)
    )
def interactive_plot(sel_word, layer, task, epoch):
    d = troubleshooting.Diagnosis(code_name)
    d.eval(testset_name, task=task, epoch=epoch)
    d.set_target_word(sel_word)
    print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
    return d.plot_one_layer(layer)
